In [3]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop


from keras import backend as K
from keras import layers
import numpy as np
import tensorflow as tf

class NormalDensity(layers.Layer):

    def __init__(self, output_dim, batch_size, **kwargs):
        self.output_dim = output_dim
        self.batch_size = batch_size
        super(NormalDensity, self).__init__(**kwargs)
        
    def build(self, input_shape):
        
        #Define set of traditional weights
        self.w = self.add_weight(name='w', 
                                 shape=(input_shape[1], self.output_dim),
                                 initializer='uniform',
                                 trainable=True)
        
        #Define our plasticity coefficient
        self.alpha = self.add_weight(name='alpha', 
                              shape=(1, 1),
                              initializer='uniform',
                              trainable=True)
        
        
        #The Hebbian trace
        self.hebb = self.add_weight(name='hebb', 
                              shape=(input_shape[1], self.output_dim),
                              initializer='zeros',
                              trainable=False)
        
        #Step size will be optimized
        self.eta = self.add_weight(name='eta', 
                                      shape=(1, 1),
                                      initializer='uniform',
                                      trainable=False)
        super(NormalDensity, self).build(input_shape)


    def call(self, x):
        
        #X (layer input)     : shape(?, INPUT_DIM)
        #W                   : shape(INPUT_DIM, OUTPUT_DIM)
        #hebb                : shape(INPUT_DIM, OUTPUT_DIM)
        #Y (layer output)    : shape(?, OUTPUT_DIM)
        #ETA                 : scalar (one per layer)
                
        y = K.dot(x, self.w)
        plastic_y = self.alpha * (K.dot(x, self.hebb))   
        model_out = K.maximum(0.0, y + plastic_y)
        
        #Hebbian update - option 1
        self.hebb = self.eta * K.dot(x, model_out) + (1 - self.eta) * self.hebb
        
        #Hebbian update - option 2
        #self.hebb +=self.eta * K.dot(model_out, (x - (K.dot(model_out, self.hebb))))

        return model_out
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)




batch_size = 128
num_classes = 10
epochs = 16

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

#Our layer using DP
#model.add(NormalDensity(50, batch_size, input_shape=(784,)))

#Traditional dense layer
model.add(Dense(50, input_shape=(784,)))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 50)                39250     
_________________________________________________________________
activation_4 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                510       
_________________________________________________________________
activation_5 (Activation)    (None, 10)                0         
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/16
60000/60000 [==============================] - 5s 79us/step - loss: 0.4310 - acc: 0.8832 - val_loss: 0.2645 - val_acc: 0.9240
Epoch 2/16
60000/60000 [====================